In [1]:
import random
import math
import matplotlib.pyplot as plt
import numpy as np

n = 1000
pts = np.random.rand(2, n)

dist_mat = np.zeros((n, n))
for i in range(n - 1):
    for j in range(i + 1, n):
        dist_mat[i, j] = np.linalg.norm(pts[:, i] - pts[:, j])
        dist_mat[j, i] = dist_mat[i, j]

def calc_dist(tour_seq, dist_mat):
    return sum(dist_mat[tour_seq[i], tour_seq[i + 1]] for i in range(len(tour_seq) - 1)) + dist_mat[tour_seq[-1], tour_seq[0]]

def sim_anneal_tsp(n, dist_mat, init_temp=1000, max_iter=500000):
    curr_tour = list(range(n))
    random.shuffle(curr_tour)

    best_tour = curr_tour[:]
    best_dist = calc_dist(best_tour, dist_mat)

    cost_hist = []

    for iter in range(1, max_iter + 1):
        ids = random.sample(range(n), 2)
        ids.sort()

        new_tour = curr_tour[:]
        new_tour[ids[0]:ids[1] + 1] = reversed(curr_tour[ids[0]:ids[1] + 1])

        new_cost = calc_dist(new_tour, dist_mat)

        delta_E = best_dist - new_cost
        temp = init_temp / iter

        if delta_E > 0:
            p_accept = 1
        elif temp <= 0:
            p_accept = 0
        else:
            if delta_E / temp > 500:
                p_accept = 0
            elif delta_E / temp < -500:
                p_accept = 1
            else:
                p_accept = 1 / (1 + math.exp(-delta_E / temp))

        if delta_E > 0 or random.random() < p_accept:
            curr_tour = new_tour
            if new_cost < best_dist:
                best_tour = curr_tour
                best_dist = new_cost

        cost_hist.append(best_dist)

    return best_tour, best_dist, cost_hist

final_tour, final_dist, cost_hist = sim_anneal_tsp(n, dist_mat)

print("Optimal tour:", final_tour)
print("Optimal distance:", final_dist)

plt.figure(figsize=(10, 6))
plt.plot(cost_hist, label='Total Distance (Cost)', color='blue')
plt.xlabel('Iterations')
plt.ylabel('Total Distance (Cost)')
plt.title('Cost vs Iterations for Simulated Annealing (TSP)')
plt.grid(True)
plt.legend()
plt.show()